# IIC-3800 Tópicos en CC - NLP UC

- Versiones de librerías, python 3.8.10

- numpy 1.20.3
- nltk 3.7
- lime 0.2.0.1
- spacy 3.5.1
- gcsfs 2023.3.0
- protobuf 3.20.3

In [1]:
import pandas as pd

df = pd.read_csv("film_affinity.csv")
df

,critica,nota,url
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...
...,...,...,...
4795,"""Shrek"" es sin lugar a dudas una de las mejore...",9,https://www.filmaffinity.com/es/reviews/6/4945...
4796,"Muy buena e incluso diría, inteligente comedia...",8,https://www.filmaffinity.com/es/reviews/3/9420...
4797,Cuando una película consigue hacer que algo ta...,7,https://www.filmaffinity.com/es/reviews/3/9420...
4798,Una gran comedia estupida que cumple su funció...,8,https://www.filmaffinity.com/es/reviews/3/9420...


____________________________________________________________________________________________________________

## Actividad en clase

Construya un clasificador de sentimiento en **castellano** usando el dataset FilmAffinity y Spacy. Para esto haga lo siguiente:

- Cree la columna 'sentiment' a partir de **nota**. Sentiment debe considerar dos clases: **positivo** y **negativo**. Observe que nota va de 1 a 10. 
- Preprocese el texto del campo **critica** usando lo que hemos visto en clases. Note que deberá cambiar algunos procesos ya que el texto está en **castellano**.
- Particione el dataset en particiones de training/development/testing (75/15/10)
- Convierta los datos para procesarlos con Spacy (use **convert**). 
- Entrene un clasificador reproduciendo el método visto en clases.
- Evalúe el clasificador sobre la particion de test.
- Cuanto termine, me avisa para entregarle una **L (logrado)**.
- Recuerde que las L otorgan un bono en la nota final de la asignatura.


***Tiene hasta el final de la clase.***

_________________________________________________________________________________________________________________

# Solución

!python3 -m spacy download es_core_news_sm

In [2]:
import string
import re
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

nlp = spacy.load("es_core_news_sm") # a spanish-based nlp model
REGX_USERNAME = r"@[A-Za-z0-9$-_@.&+]+"

def preprocessing(text):
  text = text.lower()
  text = re.sub(REGX_USERNAME, ' ', text)
  tokens = [token.text for token in nlp(text)]
  tokens = [t for t in tokens if t not in STOP_WORDS and t not in string.punctuation and len(t) > 2]
  tokens = [t for t in tokens if not t.isdigit()]

  return " ".join(tokens)


df["text_clean"] = df["critica"].apply(preprocessing)

In [3]:
df["sentiment"] = df.apply(lambda x: 0 if int(x["nota"]) < 5  else 1, axis=1)

In [4]:
df.head()

,critica,nota,url,text_clean,sentiment
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...,gusto fracaso empezó año aves presa acaba año ...,0
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...,terrible funcionar parodia sobreactuada record...,0
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...,tradición años diciembre cine blockbuster peli...,0
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...,entiendo cara presentar película entiendo crít...,0
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...,entrega wonder woman pareció maravilla contrar...,0


In [5]:
dataset = list(df[["text_clean", "sentiment"]].sample(frac=1).itertuples(index=False, name=None))
train_data = dataset[:3600]  # 75%
dev_data = dataset[3600:4320] # 15%
test_data = dataset[4320:] # 10%

In [6]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        doc.cats["POS"] = label == 1
        doc.cats["NEG"] = label == 0
        db.add(doc)
    
    db.to_disk(outfile)
convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

In [8]:
!py -m spacy init config --lang es --pipeline textcat --optimize efficiency --force config.cfg

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: es
- Pipeline: textcat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
!py -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output model --verbose

✔ Created output directory: model
ℹ Saving to output directory: model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       34.44    0.34
  0     200         43.28       69.14    0.69
  0     400         30.99       79.57    0.80
  0     600         32.40       80.44    0.80
  0     800         29.74       79.98    0.80
  0    1000         24.75       81.60    0.82
  0    1200         26.27       82.22    0.82
  0    1400         25.88       84.27    0.84
  0    1600         20.40       86.47    0.86
  0    1800         19.32       86.24    0.86
  1    2000          4.04       86.25    0.86
  1    2200          3.56       86.11    0.86
  1    2400          4.36       86.38    

[2023-05-25 12:19:32,764] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2023-05-25 12:19:32,987] [INFO] Set up nlp object from config
[2023-05-25 12:19:32,998] [DEBUG] Loading corpus from path: dev.spacy
[2023-05-25 12:19:33,003] [DEBUG] Loading corpus from path: train.spacy
[2023-05-25 12:19:33,003] [INFO] Pipeline: ['textcat']
[2023-05-25 12:19:33,003] [INFO] Created vocabulary
[2023-05-25 12:19:33,003] [INFO] Finished initializing nlp object
[2023-05-25 12:19:42,796] [INFO] Initialized pipeline components: ['textcat']
[2023-05-25 12:19:42,816] [DEBUG] Loading corpus from path: dev.spacy
[2023-05-25 12:19:42,816] [DEBUG] Loading corpus from path: train.spacy


In [10]:
!py -m spacy evaluate ./model/model-best/ ./test.spacy

ℹ Using CPU

================================== Results ==================================

TOK                 100.00
TEXTCAT (macro F)   82.70 
SPEED               284134


=========================== Textcat F (per label) ===========================

          P       R       F
POS   83.19   81.43   82.30
NEG   82.26   83.95   83.10


======================== Textcat ROC AUC (per label) ========================

      ROC AUC
POS      0.91
NEG      0.91



In [11]:
texts = ["La pelicula es innecesariamente larga. A ratos se vuelve aburrida y es dificil de seguir.", "Me arrepiento de haber comprado en esta tienda."]
nlp = spacy.load("./model/model-best")
for text in texts:
    doc = nlp(preprocessing(text))
    print(doc.cats,  "-",  text)

{'POS': 0.2130904495716095, 'NEG': 0.7869095802307129} - La pelicula es innecesariamente larga. A ratos se vuelve aburrida y es dificil de seguir.
{'POS': 0.3088102638721466, 'NEG': 0.691189706325531} - Me arrepiento de haber comprado en esta tienda.
